In [ ]:
# install fastai
#!pip install -Uqq fastbook

In [1]:
import pandas as pd
import re  
import requests 

In [3]:
# code to preprocessing for collab model

#df_overview = pd.read_csv("./artifacts/cleaned_dfs/overview_df_cleaned_df.csv")
#df_overview.drop([df_overview.columns[0]], axis=1, inplace=True)
#df_overview.dropna(inplace=True)
#df_movies = df_overview.merge(df_movies, on='movie_name', how='left')
#df_ratings = df_ratings.merge(df_movies, on=['movie_id'], how='left')
#df_ratings.drop(['genres','movie_id', 'overview'], axis=1, inplace=True)

df_ratings = pd.read_csv("./artifacts/cleaned_dfs/ratings_cleaned_df.csv")
df_movies = pd.read_csv("./artifacts/cleaned_dfs/movies_cleaned_df.csv")

df_ratings = df_ratings.drop('Unnamed: 0' , axis = 1)
df_movies = df_movies.drop('Unnamed: 0' , axis = 1)

df_ratings.columns = ['UserID','MovieID','Rating','Timestamp']
df_ratings.drop(['Timestamp'], axis=1, inplace=True)


df_ratings = df_ratings[df_ratings['MovieID'].isin(df_movies['MovieID'])]
df_ratings = df_ratings.merge(df_movies , on = 'MovieID' , how = 'left')

df_ratings = df_ratings[['UserID','Title','Rating']]


In [4]:
df_ratings.head()

,UserID,Title,Rating
0,1,One Flew Over the Cuckoo's Nest,5
1,1,James and the Giant Peach,3
2,1,My Fair Lady,3
3,1,Erin Brockovich,4
4,1,"Bug's Life, A",5


In [ ]:
learn = load_learner('collab_model2.pkl')
# get dataloaders, before using the learner for inference as the learner is empty after loading
dls = CollabDataLoaders.from_df(df_ratings, item_name='Title')
# after loading we must add the data back in the learner
learn.dls = dls

In [3]:
## code to search upcoming movies

import requests 
import datetime 

## code to search upcoming movies

def search_upcoming_movie():
  api_key = "012269e221ffedf63bb0c20f32df1127"
  base_url = f"https://api.themoviedb.org/3/movie/upcoming?api_key={api_key}&language=en-US&page=1"
  
  try:
    response = requests.get(base_url , verify = False)
    response.raise_for_status() #raise an error if API call fails

    data = response.json()
    data = data['results']

    return data 
  
  except requests.exceptions.RequestException as e:
    print(f"an error occurred: {e}")

data = search_upcoming_movie()

if data : 
  for movie in data : 
    if datetime.datetime.strptime(movie['release_date'] , "%Y-%m-%d").date() >= datetime.date.today() : 
      print(movie['title'])
      print(movie['poster_path'])
      print(movie['release_date'])

  

an error occurred: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/movie/upcoming?api_key=012269e221ffedf63bb0c20f32df1127&language=en-US&page=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F2D1B600E0>, 'Connection to api.themoviedb.org timed out. (connect timeout=None)'))
